In [1]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install fsspec

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install bs4

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install scikit-learn-intelex


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:

pip install onednn-cpu-gomp

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:

#0-offensive 1-Hate 2-Neither

import numpy as np
import pandas as pd
import pickle
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


data=pd.read_csv('data/labeled_data.csv')
#print(data)
data = data.dropna()
#X = ['hi', 'idiot', 'u are black']
#y = [2, 0, 1]
X=data['tweet']
y=data['class']
#print('X',X)
#print(y)
def create_and_train_model(X, y):
    
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(X)
    sequences = tokenizer.texts_to_sequences(X)
    X_padded = pad_sequences(sequences)

    
    X_train, X_test, y_train, y_test = train_test_split(X_padded, y_encoded, test_size=0.2, random_state=42)

    
    model = Sequential()
    model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=X_padded.shape[1]))
    model.add(LSTM(units=50, return_sequences=True))
    model.add(LSTM(units=50))
    model.add(Dense(units=64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=len(np.unique(y_encoded)), activation='softmax'))  

    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    
    model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

    
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f"Test Accuracy: {accuracy}")

    model.save('D:/content_moderation_model.h5')
    print("Model saved.")

    
    with open('D:/tokenizer.pkl', 'wb') as file:
        pickle.dump(tokenizer, file)

    with open('D:/ label_encoder.pkl', 'wb') as file:
        pickle.dump(label_encoder, file)

    print("Tokenizer and LabelEncoder saved.")

    

    return model, tokenizer, label_encoder

create_and_train_model(X,y)



2024-03-08 22:23:45.585143: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-08 22:23:45.620495: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-08 22:23:45.620551: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-08 22:23:45.621656: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-08 22:23:45.627633: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-08 22:23:45.628837: I tensorflow/core/platform/cpu_feature_guard.cc:1

Epoch 1/5
778/778 [==============================] - 86s 108ms/step - loss: 0.3716 - accuracy: 0.8794 - val_loss: 0.2378 - val_accuracy: 0.9177
Epoch 2/5
 51/778 [>.............................] - ETA: 1:13 - loss: 0.1694 - accuracy: 0.9491

In [ ]:
print('hello')

In [5]:
def predict_with_model(model, tokenizer, label_encoder, input_text):
    input_sequence = tokenizer.texts_to_sequences([input_text])
    padded_sequence = pad_sequences(input_sequence, maxlen=model.input_shape[1])

   
    predicted_probabilities = model.predict(padded_sequence)
    predicted_class = np.argmax(predicted_probabilities)
    predicted_label = label_encoder.inverse_transform([predicted_class])[0]
    confidence = predicted_probabilities[0][predicted_class]

    if confidence <= 0.6:
        predicted_label=2
    


    
    print("\nInput Text:")
    print(f"- {input_text}")

    print("\nPredicted Probabilities:")
    print(predicted_probabilities)

    print("\nPredicted Class:")
    print(predicted_label)
    return predicted_label

def load_saved_model():
    
    loaded_model = load_model('D:/content_moderation_model.h5')

    
    with open('D:/tokenizer.pkl', 'rb') as file:
        loaded_tokenizer = pickle.load(file)

    with open('D:/ label_encoder.pkl', 'rb') as file:  # Corrected file name
        loaded_label_encoder = pickle.load(file)

    print("Model, Tokenizer, and LabelEncoder loaded.")
    return loaded_model, loaded_tokenizer, loaded_label_encoder


data=pd.read_csv('data/labeled_data.csv')
#print(data)
#X = ['hi', 'idiot', 'u are black']
#y = [2, 0, 1]
X=data['tweet']
y=data['class']
#print('X',X)
#print(y)
def input_model(content):
    #print(content)
    prediction=[]
    train_model=False
    if train_model:
        trained_model, tokenizer, label_encoder = create_and_train_model(X, y)
    else:
        trained_model, tokenizer, label_encoder = load_saved_model()

    result_string = content.replace('"', '').replace("'", '')

    soup = BeautifulSoup(content, 'html.parser')
    p_tag = soup.find_all('p')
    #print("p_tag",p_tag)
    for i in p_tag:
        print(i)
        cleaned_text = i.get_text(separator=' ', strip=True) if i else ''
        #print(cleaned_text)
        a=predict_with_model(trained_model, tokenizer, label_encoder, cleaned_text)
        prediction.append(a)
        print("Predictions",prediction)
    
    return prediction
 
print(input_model('<p><img alt="" src="/media/uploads/2023/12/04/test2-94.jpg" style="height:200px; width:150px" /></p><p>You are nothing but a waste of space in this bloody world...women belong in the kitchen you slut idiot i hate u so much you black people dont belong here women are a peice of shit who belong in th bloody kitchen hell dammit shit bloody losers</p>'))

Model, Tokenizer, and LabelEncoder loaded.
<p><img alt="" src="/media/uploads/2023/12/04/test2-94.jpg" style="height:200px; width:150px"/></p>
1/1 [==============================] - 1s 605ms/step

Input Text:
- 

Predicted Probabilities:
[[0.0254811  0.05050526 0.9240137 ]]

Predicted Class:
2.0
Predictions [2.0]
<p>You are nothing but a waste of space in this bloody world...women belong in the kitchen you slut idiot i hate u so much you black people dont belong here women are a peice of shit who belong in th bloody kitchen hell dammit shit bloody losers</p>
1/1 [==============================] - 0s 26ms/step

Input Text:
- You are nothing but a waste of space in this bloody world...women belong in the kitchen you slut idiot i hate u so much you black people dont belong here women are a peice of shit who belong in th bloody kitchen hell dammit shit bloody losers

Predicted Probabilities:
[[8.83877464e-03 9.91044521e-01 1.16756644e-04]]

Predicted Class:
1.0
Predictions [2.0, 1.0]
[2.0,